In [2]:
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import darts
from darts.utils.statistics import check_seasonality, plot_acf, stationarity_tests
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.dataprocessing.transformers.boxcox import BoxCox
from darts.dataprocessing.transformers.diff import Diff
from darts.utils.statistics import plot_hist
from darts.models import LightGBMModel, XGBModel, LinearRegressionModel, TFTModel, NHiTSModel
from darts.metrics import smape, mape, mase, mse, rmse, r2_score, mae
from darts.dataprocessing.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler   
from darts.dataprocessing.transformers.scaler import Scaler

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import torch

from utils import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


In [2]:
# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"

Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [3]:
# run parameters

dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')


config_dict = {
    'spatial_scale': 'county',
    'temp_resolution': '60',
    'location': 'New_York',
    'horizon': 24, # in hours
    'n_lags': 24, # timesteps lookback
}
    
spatial_scale = 'county'
temp_resolution = '60'
location = 'New_York'
horizon = 24 # in hours


n_lags = 24 # timesteps lookback
n_ahead = int(horizon * 60//int(temp_resolution)) # timesteps ahead

list_sklearn_models= [
                        LinearRegressionModel,
                        LightGBMModel,
                        XGBModel,
                        ]

In [5]:
# Loading Data
df_train = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/train')
df_val = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/val')
df_test = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/test')
df_cov_train = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/train_weather')
df_cov_val = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/val_weather')
df_cov_test = pd.read_hdf(os.path.join(dir_path, f'{spatial_scale}_data.h5'), key=f'{temp_resolution}min/{location}/test_weather')

FileNotFoundError: File /Users/nikolaushouben/Desktop/WattCast/bin/data/clean_data/county_data.h5 does not exist

In [6]:
# to darts format

# into darts format

ts_train = darts.TimeSeries.from_dataframe(df_train)
ts_val = darts.TimeSeries.from_dataframe(df_val)
ts_test = darts.TimeSeries.from_dataframe(df_test)

# Covariates
ts_cov_train = darts.TimeSeries.from_dataframe(df_cov_train)
ts_cov_val = darts.TimeSeries.from_dataframe(df_cov_val)
ts_cov_test = darts.TimeSeries.from_dataframe(df_cov_test)



In [10]:
# pipeline

# Target pipeline
pipeline = Pipeline(
                    [
                    MissingValuesFiller(), 
                    Scaler(MinMaxScaler()),
                    ]
                     )


ts_train_piped = pipeline.fit_transform(ts_train)
ts_val_piped = pipeline.transform(ts_val)
ts_test_piped = pipeline.transform(ts_test)

# Weather Pipeline
pipeline_weather = Pipeline([MissingValuesFiller() ,Scaler(RobustScaler())])
ts_train_weather_piped = pipeline_weather.fit_transform(ts_cov_train)
ts_val_weather_piped = pipeline_weather.transform(ts_cov_val)
ts_test_weather_piped = pipeline_weather.transform(ts_cov_test)

In [11]:
# model settings 


trg_train, covs_train = make_index_same(ts_train_piped, ts_train_weather_piped)
trg_train_inversed = pipeline.inverse_transform(trg_train, partial=True) # inverse transform the target, we need the original values for the evaluation

trg_test, covs_test = make_index_same(ts_test_piped, ts_test_weather_piped)
trg_test_inversed = pipeline.inverse_transform(trg_test, partial=True) # inverse transform the target, we need the original values for the evaluation

encoders = {
            "cyclic": {"future": ["hour"]}, 
            "position": {"future": ["relative",]},
            "datetime_attribute": {"future": ["dayofweek", "week"]}
            }

In [15]:
# instantiate models
liklihood = None

models_with_weather = make_sklearn_models(list_sklearn_models, encoders, n_lags, n_ahead, liklihood)

# Train the models with weather
models_with_weather = train_models(models_with_weather, 
                        trg_train, 
                        covs_train
                        )
model_names_with_weather = [model.__class__.__name__ + "_with_weather" for model in models_with_weather]

In [17]:
models = models_with_weather
model_names = model_names_with_weather
models_dict = {model_name: model for model_name, model in zip(model_names, models)}


In [19]:
# Generating Historical Forecasts for each model
ts_predictions_per_model = {}
historics_per_model = {}

for model_name, model in models_dict.items():
    covs_use = None if model_name.endswith("o_weather") else covs_test
    historics = model.historical_forecasts(trg_test, 
                                        future_covariates= covs_use,
                                        start=ts_test_piped.get_index_at_point(n_lags),
                                        verbose=True,
                                        stride=n_ahead // 8, 
                                        forecast_horizon=n_ahead, 
                                        retrain=False, 
                                        last_points_only=False, # leave this as False unless you want the output to be one series, the rest will not work with this however
                                        #num_samples=30 # only enable when using quantile regression likelihood
                                        )
    
    historics_per_model[model_name] = historics # storing the forecasts in batches of the forecasting horizon, for plot 1
    ts_predictions = ts_list_concat(historics) # concatenating the batches into a single time series for plot 2
    ts_predictions_inverse = pipeline.inverse_transform(ts_predictions, partial=True) # inverse transform the predictions, we need the original values for the evaluation
    ts_predictions_per_model[model_name] = ts_predictions_inverse

  0%|          | 0/5585 [00:00<?, ?it/s]

  0%|          | 0/5585 [00:00<?, ?it/s]

  0%|          | 0/5585 [00:00<?, ?it/s]

In [20]:
df_smapes_per_model = []
df_nrmse_per_model = []

for model_name, historics in historics_per_model.items():
    df_list = get_df_compares_list(historics, trg_test)
    means_per_timestep = pd.concat(df_list, axis=1).filter(like = "gt").mean(axis=1).values # mean of the ground truth values on a timestep basis
    diffs = get_df_diffs(df_list)
    df_smapes = abs(diffs).mean(axis =1) / means_per_timestep # mean of the relative errors on a timestep basis
    df_smapes.columns = [model_name]
    df_nrmse = np.square(diffs).mean(axis =1) / means_per_timestep # mean of the relative errors on a timestep basis
    df_nrmse.columns = [model_name]

    df_smapes_per_model.append(df_smapes)
    df_nrmse_per_model.append(df_nrmse)

df_smapes_per_model = pd.concat(df_smapes_per_model, axis=1)
df_smapes_per_model.columns = model_names
df_nrmse_per_model = pd.concat(df_nrmse_per_model, axis=1)
df_nrmse_per_model.columns = model_names

In [ ]:
df_smapes_per_model.plot(figsize=(10,10))
plt.xlabel('Horizon')
plt.ylabel('MAPE [%]')
plt.legend(loc = 'lower right', ncol = 2)
plt.title(f"Plot I a): Mean Absolute Percentage Error of the Historical Forecasts in {location}")

df_nrmse_per_model.plot(figsize=(10,10))
plt.xlabel('Horizon')
plt.ylabel('nRMSE [%]')
plt.legend(loc = 'lower right', ncol = 2)
plt.title(f"Plot I b): Normalised RMSE of the Historical Forecasts in {location}")